In [3]:
import os
import sys
import glob
import cv2
import matplotlib.pyplot as plt # 주피터 환경에서 이미지 열어주는거
import numpy as np

In [4]:
# 이미지를 전부 읽어와서 하나로 합친다.
train_images = []

image = cv2.imread('faces_training/face01.pgm', cv2.IMREAD_GRAYSCALE)
train_images.append(image)

length, height = image.shape
image = image.reshape((1, length * height * 1))

for i in range(2,10):
    x = cv2.imread(f'faces_training/face0{i}.pgm', cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((1, length * height * 1))
    image = np.concatenate([image, temp], axis=0)
    
for i in range(10,40):
    x = cv2.imread(f'faces_training/face{i}.pgm', cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((1, length * height * 1))
    # print(temp.shape)
    image = np.concatenate([image, temp], axis=0)

# image = np.concatenate([x,y,z], axis=0)
print(image.shape)

(39, 32256)


In [1]:
print("######### Step1 #########\n")
    
def computeDimensions(img, percent=0.9):
    
    # computing eigenvalues and eigenvectors of covariance matrix
    matrix = (img - np.mean(img.T, axis=1)).T # subtract the mean (along columns)

    [eigvals, eigvec] = np.linalg.eig(np.cov(img))
    
    whole_nom_of_pc = np.size(eigvec, axis=0)
    
    eigvals = sorted(eigvals)
    eigvals = eigvals[::-1]
    
    ssum = 0
    numPc=0
    sumvar = sum(eigvals)
        
    for i in range(whole_nom_of_pc):
        ssum += eigvals[i]
        numPc += 1
        # print(f"현재 고유 값{i} : {eigvals[i]}")
        if(ssum/sumvar >= percent):
            print("확률 : ",ssum/sumvar)
            break
    
    print("전체 차원 수 ", whole_nom_of_pc)
    print("선택된 차원 수 ", numPc)    
    return numPc
    
perc = 0.93
numPc = computeDimensions(image, perc)
# print(f"nput Percentage: {sys.argv[1]}")
print(f"Input Percentage: {perc}\n")
print(f"Selected Dimansion : {numPc}\n")
    

######### Step1 #########



NameError: name 'image' is not defined

In [5]:
# for step2,
# reconsturct 함수
def princomp(img, pc_limit=0):

    # computing eigenvalues and eigenvectors of covariance matrix
    matrix = (img - np.mean(img.T, axis=1)).T # subtract the mean (along columns)
    
    [eigvals,eigvecs] = np.linalg.eig(np.cov(matrix))
    
    whole_num_pc = np.size(eigvecs,axis=1)

    sortEigvals = sorted(eigvals) # 고유값을 정렬한다.
    sortEigvals = sortEigvals[::-1]       # in ascending order. 순서를 높은게 앞에 오도록 변경
    # sorting eigenvectors according to the sorted eigenvalues
    # 고유값이 정렬된 순서에 따라 대응하는 고유벡터들도 정렬한다.
    # coeff = coeff[:,sortEigvals]
    
    # eigvals = eigvals[sortEigvals] # sorting eigenvalues
    if pc_limit < whole_num_pc and pc_limit >= 0:
        eigvecs = eigvecs[:,range(pc_limit)] # 입력된 pc 개수에 따라 선택한다.
        eigvals = eigvals[:pc_limit]
    
    print(eigvecs.shape)
    score = np.dot(eigvecs.T,matrix) # projection of the data in the new space
    final = np.dot(eigvecs, score).T + np.mean(img, axis=0)
    return eigvecs,score,eigvals, final

In [7]:
final_list = []

for n in range(39):
    img = (train_images[n]-np.mean(train_images[n].T,axis=1)) # subtract the mean (along columns)
    
    eigenvector, score, eigenvalue, final = princomp(train_images[n], numPc)
    final_list.append(np.abs(final))

    row, col = train_images[n].shape
    summ = 0
    for i in range(row):
        for j in range(col):
            s = (train_images[n][i][j]-np.abs(final[i][j]))**2
            summ += s # 전체 합.
    re_error = summ/(row*col) # 전체합을 n으로 나누고 복소수를 float으로 전환
    print(f"Reconstructed error :{round(re_error, 4)}") # Reconstruction error 소숫점 4자리까지 보여짐

# final_list[0] 은 face01의 리컨스트럭션 이미지다.

(168, 21)
Reconstructed error :19.5209
(168, 21)
Reconstructed error :12.9484
(168, 21)
Reconstructed error :20.0204
(168, 21)
Reconstructed error :10.7292
(168, 21)
Reconstructed error :24.5289
(168, 21)
Reconstructed error :21.8178
(168, 21)
Reconstructed error :12.7641
(168, 21)
Reconstructed error :16.3334
(168, 21)
Reconstructed error :14.7948
(168, 21)
Reconstructed error :25.8667
(168, 21)
Reconstructed error :15.7
(168, 21)
Reconstructed error :18.2981
(168, 21)
Reconstructed error :16.843
(168, 21)
Reconstructed error :16.2112
(168, 21)
Reconstructed error :18.4964
(168, 21)
Reconstructed error :17.4844
(168, 21)
Reconstructed error :19.0333
(168, 21)
Reconstructed error :13.5554
(168, 21)
Reconstructed error :15.194
(168, 21)
Reconstructed error :10.6316
(168, 21)
Reconstructed error :20.1433
(168, 21)
Reconstructed error :26.6685
(168, 21)
Reconstructed error :17.1372
(168, 21)
Reconstructed error :13.1527
(168, 21)
Reconstructed error :19.7814
(168, 21)
Reconstructed error 

In [114]:
test01 = cv2.imread('faces_test/test02.pgm', cv2.IMREAD_GRAYSCALE)

print(len(final_list))
print(final_list[0].shape)
print("######### Step3 #########")
row, col = test01.shape
for a in range(1,6):
    distances = []
    test01 = cv2.imread(f'faces_test/test0{a}.pgm', cv2.IMREAD_GRAYSCALE)
    for n in range(39): 
        sum_of=0
        for r in range(row):
            for c in range(col):
                sum_of += (test01[r][c]-final_list[n][r][c])**2
        distances.append(np.sqrt(sum_of))
    found = distances.index(min(distances)) + 1
    if found >= 10 : print(f"test0{a}.pgm ==> face{found}.pgm")
    else : print(f"test0{a}.pgm ==> face0{found}.pgm")



39
(192, 168)
######### Step3 #########
test01.pgm ==> face11.pgm
test02.pgm ==> face07.pgm
test03.pgm ==> face19.pgm
test04.pgm ==> face23.pgm
test05.pgm ==> face35.pgm


In [58]:
test01 = cv2.imread('faces_test/test01.pgm', cv2.IMREAD_GRAYSCALE)

eigenvector, score, eigenvalue, tf = princomp(test01, numPc)

print(len(final_list))
print(final_list[0].shape)

row, col = test01.shape
distances = []
for n in range(39): 
    sum_of=0
    for r in range(row):
        for c in range(col):
            sum_of += (tf[r][c]-final_list[n][r][c])**2
    distances.append(np.sqrt(sum_of))

print(distances)
found = distances.index(min(distances)) + 1
print(found)



39
(192, 168)
[6030.343623650764, 8686.624613345479, 7049.724783722562, 7785.366708591398, 9736.16659875407, 7583.831652989635, 7211.932436206916, 7130.609198878587, 7304.662758109978, 9392.386461216422, 4843.56766795864, 7771.264560539638, 7239.084839066912, 13902.471520014516, 8648.765431902888, 8677.778649889422, 7474.013556074829, 6748.721731687042, 7238.889155618456, 6597.873976635354, 7959.520382950866, 8794.650621535648, 7458.113454781669, 6586.942161946845, 5798.237197539785, 7645.051846357443, 9762.28068056328, 8843.465330318462, 10763.66166374329, 7881.382364811592, 7154.975568493266, 8894.844744606997, 6953.047814597506, 7156.52144176358, 9405.456996341132, 7560.4912311799235, 6941.568157868283, 6853.243714646259, 8006.818382629144]
11
